In [ ]:
from notebook_prelude import *
import experiments
from experiments import task_helper, task_runner
from preprocessing import preprocessing

In [ ]:
X, Y = dataset_helper.get_dataset('imsdb')
X = [preprocessing.preprocess(x) for x in X]

In [ ]:
word_counts = [len(x.split()) for x in X]

In [ ]:
fig, ax = plt.subplots()
df = pd.DataFrame(word_counts, columns = ['num_words'])
df.num_words.plot(kind='hist', bins=60)
ax.axvline(df.num_words.median(), c='red')
ax.text(df.num_words.median(), -10, df.num_words.median(), fontdict=dict(horizontalalignment='center'))
ax.grid('off')
ax.set_xlabel('# words per document')

In [ ]:
task = experiments.get_filtered_tasks(
    task_type='graph_combined',
    dataset='ng20',
    graph_type_filter=TYPE_CONCEPT_MAP
)[0]
X_, Y_, estimator_, params_ = task.fn()

In [ ]:
X, Y, estimator, params = np.copy(X_), np.copy(Y_), sklearn.base.clone(estimator_), dict(**params_)
params = task_helper.add_classifier_to_params(params)
print(params)
assert False
#params['feature_extraction__fast_wl__ignore_label_order'] = [True, False]
params['features__fast_wl_pipeline__feature_extraction__feature_extraction__fast_wl__ignore_label_order'] = [True, False]

X_train, X_test, Y_train, Y_test, train_i, test_i = task_runner.train_test_split(
    X,
    Y,
    test_size=0.15,
    random_state=42,
    is_precomputed=False
)

cv = sklearn.model_selection.StratifiedKFold(
    n_splits=3,
    random_state=42,
    shuffle=True
)


gscv = sklearn.model_selection.GridSearchCV(estimator, param_grid=params, scoring='f1_macro', cv=cv, verbose=10)
gscv_result = gscv.fit(X_train, Y_train)

In [ ]:
gscv_result.cv_results_['mean_test_score']